# **Machine Learning**

### Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## **Consideraciones Indiciales para la Implementación de un Sistema de Recomendación de Películas**

A continuación se presentarán diferentes modelos de Sistema de Recomendación de Películas basados en columnas analizadas en el EDA y siguiendo enfoques comúnmente utilizados. Debido a las limitaciones computacionales de Google Colab, se adaptarán algunos parámetros y métodos, y el código se presentará estratégicamente separado en diferentes celdas en lugar de en una única función.

Es importante tener en cuenta que la selección final del modelo que se utilizará en este proyecto estará sujeta a las limitaciones del plan de desarrollador gratuito de Render que ofrece 512 MB de memoria RAM. Este servicio se utilizará para implementar el modelo.

## **Ingesta de Datos limpios**

In [ ]:
# leer los datos que fueron previamente analizados en el EDA
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PI_MLOps/movies_clean.csv')

In [ ]:
# Mostrar información del número de columnas, filas, tipos de datos y valores faltantes presentes en "df"
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45346 entries, 0 to 45345
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                45346 non-null  int64  
 1   genres                44900 non-null  object 
 2   id                    45346 non-null  int64  
 3   original_language     45335 non-null  object 
 4   overview              45346 non-null  object 
 5   popularity            45346 non-null  float64
 6   production_companies  33557 non-null  object 
 7   production_countries  39138 non-null  object 
 8   release_date          45346 non-null  object 
 9   revenue               45346 non-null  float64
 10  runtime               45327 non-null  float64
 11  status                45266 non-null  object 
 12  tagline               20383 non-null  object 
 13  title                 45346 non-null  object 
 14  vote_average          45346 non-null  float64
 15  vote_count         

In [ ]:
# Seleccionamos solo las filas duplicadas en la columna 'title'
duplicados = df[df.duplicated(['title'], keep=False)]

# Ordenamos el DataFrame por el valor de la columna 'title'
duplicados = duplicados.sort_values('title')

duplicados[['title', 'vote_count', 'release_year', 'production_companies']]

,title,vote_count,release_year,production_companies
24374,10 Minutes,7.0,2002,NaN
42590,10 Minutes,2.0,2014,NaN
1159,12 Angry Men,2130.0,1957,"United Artists,Orion-Nova Productions"
15187,12 Angry Men,59.0,1997,MGM Television
32730,12 Chairs,24.0,1971,Mosfilm
...,...,...,...,...
11638,Zodiac,2080.0,2007,"Paramount Pictures,Warner Bros.,Phoenix Pictures"
11163,Zoom,140.0,2006,"Columbia Pictures,Revolution Studios"
37819,Zoom,25.0,2015,"Rhombus Media,O2 Filmes"
5756,Zulu,137.0,1964,Diamond Films


Hay títulos duplicados de películas que fueron lanzadas por diferentes compañías productoras en años distintos y que de hecho presentan votaciones distintas. Esto es un dato importante a la hora de construir el modelo de Sistema de Recomendación de Películas.

## **Modelo de Recomendación de Película basado en contenido**

Un modelo de sistema de recomendación de películas basado en contenido es un algoritmo que analiza las características de las películas (como el género, el reparto, la sinopsis, etc.) para recomendar películas similares a las que un usuario ha disfrutado en el pasado.

Este tipo de modelo utiliza técnicas de procesamiento de lenguaje natural (NLP) y aprendizaje automático para analizar los datos de las películas y crear un perfil de cada película, que se utiliza para calcular la similitud entre las películas.

### Modelo 1

Considerando únicamente la descripciones de las películas ```overview```

Esta es una técnica de recomendación útil para encontrar películas similares a una película de entrada utilizando la información de texto de la descripción de cada película.

Para este modelo se usara la técnica TF-IDF para crear una representación numérica de la información de texto de la descripción de cada película. La representación numérica se utiliza para calcular la similitud coseno entre cada par de películas en el dataset y, a continuación, se seleccionan las películas más similares a una película de entrada como recomendaciones.



Campos a utilizar ```title```, ```overview```

In [ ]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo1 = df[['title', 'overview_clean']].copy()

In [ ]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_1 = TfidfVectorizer(stop_words="english", ngram_range = (1, 2))
# Aplicar la transformación TF-IDF al texto contenido en la columna "overview" del dataframe 'modelo1'
tfidf_matriz_1 = tfidf_1.fit_transform(modelo1['overview_clean'])

In [ ]:
def get_recomendacion_m1(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo1'
  indices = pd.Series(modelo1.index, index=modelo1['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se devuelve un mensaje de error
  if idx is None:
    return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo1.duplicated(['title']).any():
    primer_idx = modelo1[modelo1['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_1[idx], tfidf_matriz_1).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo1.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [ ]:
# Prueba #1
get_recomendacion_m1('The Dark Knight Rises')

['The Dark Knight',
 'Batman Forever',
 'Batman',
 'Batman Returns',
 'Batman: Under the Red Hood']

In [ ]:
# Prueba #2
get_recomendacion_m1('The Avengers')

['The Work and the Glory',
 "Sir Arne's Treasure",
 'Monty Python and the Holy Grail',
 'A Beast at Bay',
 'Enigma']

In [ ]:
# Prueba #3
get_recomendacion_m1('Wonder Woman')

['Wide Eyed and Legless',
 'Trevor Noah: The Daywalker',
 'The Machinist',
 'Diana',
 'Our Dancing Daughters']

In [ ]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m1('Titanic')

['Grantham and Rose',
 'Take Care of the Women!',
 'The Legend of 1900',
 'Raise the Titanic',
 'Beyond the Poseidon Adventure']

**Conclusión:**

El hecho de que el modelo pueda identificar correctamente que "The Dark Knight Rises" es una película de Batman y recomendar otras películas similares de Batman es un buen indicio de que el modelo está funcionando adecuadamente para ciertos casos. Sin embargo, el hecho de que no pueda hacer recomendaciones precisas para películas como "The Avengers" y "Titanic" sugiere que el modelo puede tener limitaciones importantes en su capacidad para hacer recomendaciones específicas y personalizadas.

Una de las principales limitaciones del modelo es que no tiene en cuenta otras características importantes como los actores, los directores y el género de las películas. Estos son factores importantes que pueden influir en las preferencias de los usuarios y, por lo tanto, es probable que limiten la capacidad del modelo para hacer mejores recomendaciones.


### Modelo 2

Para este caso agregaremos un atributo categórico más ```overview``` y ```genres```

In [ ]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo2 = df[['title', 'overview', 'genres']].copy()
# Se realiza un preprocesamiento en la columna "genres" para separar los géneros y convertirlos en palabras individuales
modelo2['genres'] = modelo2['genres'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [ ]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_2 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Incluir la columna 'vote_count' en la concatenación de columnas
modelo2['concatenado'] = modelo2['overview'] + ' ' + modelo2['genres']
# Aplicar la transformación TF-IDF al texto contenido en las columnas "overview" y "genres" del dataframe 'modelo2'
tfidf_matriz_2 = tfidf_2.fit_transform(modelo2['concatenado'])

In [ ]:
def get_recomendacion_m2(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo2'
  indices = pd.Series(modelo2.index, index=modelo2['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se devuelve un mensaje
  if idx is None:
    return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo2.duplicated(['title']).any():
    primer_idx = modelo2[modelo2['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_2[idx], tfidf_matriz_2).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo2.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [ ]:
# Prueba #1
get_recomendacion_m2('The Dark Knight Rises')

['The Dark Knight',
 'Batman Forever',
 'Batman',
 'Batman Returns',
 'Batman: Under the Red Hood']

In [ ]:
# Prueba #2
get_recomendacion_m2('The Avengers')

['The Work and the Glory',
 'Monty Python and the Holy Grail',
 "Sir Arne's Treasure",
 'Prince of Persia: The Sands of Time',
 'Sir Henry at Rawlinson End']

In [ ]:
# Prueba #3 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m2('Wonder Woman')

['Trevor Noah: The Daywalker',
 'The Machinist',
 'Wide Eyed and Legless',
 'Diana',
 'The Sheik']

In [ ]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m2('Titanic')

['Grantham and Rose',
 'The Legend of 1900',
 'Raise the Titanic',
 'Dustbin Baby',
 '2103: The Deadly Wake']

**Conclusión:**

En comparación con el modelo anterior, el modelo actual muestra una mejora en el orden de las recomendaciones. Sin embargo, aún existen oportunidades para mejorar la precisión y personalización de las recomendaciones.

Para abordar esta limitación, se explorará la posibilidad de agregar más características resaltantes al modelo, como los actores, directores y géneros de las películas. Estas características pueden ser importantes para los usuarios al elegir películas y pueden ayudar al modelo a hacer recomendaciones más precisas y personalizadas.

### Modelo 3

Se experimentará con las columnas anteriores sumandole el atributo de ```director```

In [ ]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo3 = df[['title', 'overview_clean', 'genres', 'director']].copy()
# Se realiza un preprocesamiento en la columna "genres" para separar los géneros y convertirlos en palabras individuales
modelo3['genres'] = modelo3['genres'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "director" para separar los géneros y convertirlos en palabras individuales
modelo3['director'] = modelo3['director'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [ ]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_3 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Aplicar la transformación TF-IDF al texto contenido en las columnas "overview_clean", "genres" y "director" del dataframe 'modelo3'
tfidf_matriz_3 = tfidf_3.fit_transform(modelo3['overview_clean'] + ' ' + modelo3['genres'] + ' ' + modelo3['director'])

In [ ]:
def get_recomendacion_m3(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo3'
  indices = pd.Series(modelo3.index, index=modelo3['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se devuelve un mensaje de error
  if idx is None:
    return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo3.duplicated(['title']).any():
    primer_idx = modelo3[modelo3['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_3[idx], tfidf_matriz_3).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo3.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [ ]:
# Prueba #1
get_recomendacion_m3('The Dark Knight Rises')

['The Dark Knight',
 'Batman Forever',
 'Batman Returns',
 'Batman',
 'Batman Begins']

In [ ]:
# Prueba #2
get_recomendacion_m3('The Avengers')

['The Work and the Glory',
 'The Right Kind of Wrong',
 'Benny & Joon',
 'Diabolique',
 "Sir Arne's Treasure"]

In [ ]:
# Prueba #3 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m3('Wonder Woman')

['DC Showcase: Catwoman',
 'Green Lantern: First Flight',
 'Wide Eyed and Legless',
 'Green Lantern: Emerald Knights',
 'Superman: Doomsday']

In [ ]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m3('Titanic')

['Ghosts of the Abyss',
 'Grantham and Rose',
 'Titanic',
 'The Legend of 1900',
 'Deadly Voyage']

**Conclusión:**

Al agregar la variable director, los resultados comienzan a presentar mejorias especialmente en las recomendaciones para Titanic y Wonder Woman. Veamos que sucede si se agregar una variable más

### Modelo 4

Se finalizará con las columnas anteriores sumandole el atributo de ```director```

In [ ]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo4 = df[['title', 'actors', 'overview_clean', 'genres', 'director']].copy()
# Se realiza un preprocesamiento en la columna "genres" para separar los géneros y convertirlos en palabras individuales
modelo4['genres'] = modelo4['genres'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "director" para separar los géneros y convertirlos en palabras individuales
modelo4['director'] = modelo4['director'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "actors" para separar los géneros y convertirlos en palabras individuales
modelo4['actors'] = modelo4['actors'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [ ]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_4 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Aplicar la transformación TF-IDF al texto contenido en las columnas "overview_clean", "genres", "actors" y "director" del dataframe 'modelo4'
tfidf_matriz_4 = tfidf_4.fit_transform(modelo4['overview_clean'] + ' ' + modelo4['genres'] + ' ' + modelo4['director'] + ' ' + modelo4['actors'])

In [ ]:
def get_recomendacion_m4(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo4'
  indices = pd.Series(modelo4.index, index=modelo4['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se devuelve un mensaje de error
  if idx is None:
    return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo4.duplicated(['title']).any():
    primer_idx = modelo4[modelo4['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_4[idx], tfidf_matriz_4).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo4.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [ ]:
# Prueba #1
get_recomendacion_m4('The Dark Knight Rises')

['The Dark Knight',
 'Batman Begins',
 'Interstellar',
 'The Wolf of Wall Street',
 'Inception']

In [ ]:
# Prueba #2
get_recomendacion_m4('The Avengers')

['The Work and the Glory',
 'Hysterical Blindness',
 'Eddie Izzard: Unrepeatable',
 'Eddie Izzard: Circle',
 'The Frightened City']

In [ ]:
# Prueba #3 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m4('Wonder Woman')

['DC Showcase: Catwoman',
 'Superman/Batman: Apocalypse',
 'Green Lantern: Emerald Knights',
 'Green Lantern: First Flight',
 'Superman: Doomsday']

In [ ]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m4('Titanic')

['Aliens of the Deep',
 'Titanic: The Final Word with James Cameron',
 'Ghosts of the Abyss',
 'The Bounty',
 'The Departed']

**Conclusión:**

La inclusión del atributo "actores" ha tenido un impacto significativo en la calidad de las recomendaciones de películas. Ahora se están generando recomendaciones más personalizadas que toman en cuenta la aparición de los actores y directores en las películas, además de la descripción y el género. Por ejemplo, en el caso de The Dark Knight Rises, el modelo ya no sólo recomienda películas relacionadas con Batman, sino que también sugiere otros tipos de películas con similitudes.

Aunque esto puede considerarse una mejora con respecto a los modelos anteriores, aún hay limitaciones en la capacidad del modelo debido a la falta de información sobre el usuario y su historial de películas consumidas. Esto significa que las recomendaciones aún pueden no ser totalmente personalizadas y precisas para cada usuario.

## **Modelo de Recomendación de Películas basado en Calificaciones Ponderadas**

Para este modelo se considerará el uso de la calificación ponderada de IMDB (también conocida como "Weighted Rating" o "wr") es un método para calcular la calificación de una película que tiene en cuenta tanto la calificación promedio de la película como el número de votos que ha recibido. La fórmula para calcular la calificación ponderada de IMDB es la siguiente:

![wr](https://1.bp.blogspot.com/-IwW-xX59Hi4/YF7TyvzmM7I/AAAAAAAAdkg/34Mpp3aW5LAsy561icqkdDEsq_O2ZgI9gCLcBGAsYHQ/s762/weight-avg.png)

donde:

- R es la calificación promedio de la película (con una escala de 0 a 10).
- v es el número de votos que ha recibido la película.
- m es el número mínimo de votos requerido para que una película sea considerada en el cálculo (generalmente se fija en 25).
- C es la calificación promedio de todas las películas en la base de datos de IMDB.

In [ ]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo5 = df[['genres', 'title', 'vote_count', 'vote_average', 'director']].copy()

In [ ]:
# Procesar los datos
modelo5['genres'] = modelo5['genres'].fillna('')
modelo5['genres'] = modelo5['genres'].str.split(',')
genres = pd.Series([genre for genres in modelo5['genres'] for genre in genres]).unique()
for genre in genres:
  modelo5[genre] = modelo5['genres'].apply(lambda x: 1 if genre in x else 0)
# Agregar una nueva columna con los nombres de los directores
modelo5['directores'] = modelo5['director'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

In [ ]:
# Calificación promedio de la película
v = modelo5['vote_count']
# Número de votos que ha recibido la película
R = modelo5['vote_average']
# Número mínimo de votos requerido
m = 25
# Calcular la calificación promedio de todas las películas
C = modelo5['vote_average'].mean()
# Calcular la calificación ponderada de las películas y agregarla como una nueva columna
modelo5['calificacion_ponderada'] = ((v / (v + m)) * R) + ((m / (v + m)) * C)

In [ ]:
# Imprimir las primeras 5 filas de "credits_csv"
modelo5.head()

,genres,title,vote_count,vote_average,director,animation,comedy,family,adventure,fantasy,...,mystery,war,foreign,music,documentary,western,tvmovie,,directores,calificacion_ponderada
0,"[animation, comedy, family]",Toy Story,5415.0,7.7,John Lasseter,1,1,1,0,0,...,0,0,0,0,0,0,0,0,[John Lasseter],7.690460
1,"[adventure, fantasy, family]",Jumanji,2413.0,6.9,Joe Johnston,0,0,1,1,1,...,0,0,0,0,0,0,0,0,[Joe Johnston],6.886918
2,"[romance, comedy]",Grumpier Old Men,92.0,6.5,Howard Deutch,0,1,0,0,0,...,0,0,0,0,0,0,0,0,[Howard Deutch],6.312862
3,"[comedy, drama, romance]",Waiting to Exhale,34.0,6.1,Forest Whitaker,0,1,0,0,0,...,0,0,0,0,0,0,0,0,[Forest Whitaker],5.898388
4,[comedy],Father of the Bride Part II,173.0,5.7,Charles Shyer,0,1,0,0,0,...,0,0,0,0,0,0,0,0,[Charles Shyer],5.690429


In [ ]:
# Función para recomendar películas
def get_recomendacion_m5(titulo):
  # Obtener la fila correspondiente al título de la película que se está recomendando
  pelicula = modelo5.loc[modelo5['title'] == titulo].iloc[0]
  # Calcular la similitud entre las películas
  modelo5['similitud'] = modelo5.apply(lambda x: (x[genres] * pelicula[genres]).sum() / (x[genres].sum() + pelicula[genres].sum()), axis=1)
  # Ordenar las películas por similitud y obtener las 5 mejores recomendaciones
  recomendaciones = modelo5.sort_values('similitud', ascending=False).head(6)[1:]
  # Calcular el peso adicional para las películas con una calificación ponderada alta y agregarlo a las recomendaciones
  recomendaciones['peso_calificacion'] = recomendaciones['calificacion_ponderada'].apply(lambda x: 1 if x >= pelicula['calificacion_ponderada'] else 0)
  # Calcular el peso adicional para las películas con el mismo director que la película que se está recomendando y agregarlo a las recomendaciones
  recomendaciones['peso_director'] = recomendaciones.apply(lambda x: len(set(x['directores']).intersection(set(pelicula['directores']))) / len(set(x['directores']).union(set(pelicula['directores']))), axis=1)
  # Calcular la puntuación total para cada recomendación y obtener las 5 mejores recomendaciones
  recomendaciones['peso_total'] = recomendaciones['similitud'] + (recomendaciones['peso_calificacion'] * 0.2) + (recomendaciones['peso_director'] * 0.2)
  recomendaciones = recomendaciones.sort_values('peso_total', ascending=False).head(5)

  return recomendaciones['title'].tolist()

In [ ]:
# Prueba #2
get_recomendacion_m5('The Dark Knight Rises')

['Death Wish', 'Pusher', 'After The Storm', 'Sin Nombre', 'The Asphalt Jungle']

In [ ]:
# Prueba #2
get_recomendacion_m5('The Avengers')

['Three Strangers',
 "Twilight's Last Gleaming",
 "Don't Say a Word",
 'Survivor',
 'Dangerous Company']

In [ ]:
# Prueba #3 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m5('Wonder Woman')

['Wonder Woman',
 'DC Showcase: Catwoman',
 'Justice League: Throne of Atlantis',
 'Macross Plus: Movie Edition',
 'Green Lantern: Emerald Knights']

In [ ]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m5('Titanic')

['Él',
 'The Bridesmaid',
 'Year of the Jellyfish',
 'Live Flesh',
 'Cruel Intentions 2']

Un inconveniente con este modelo es que las recomendaciones no son tan efectivas y el tiempo de respuesta supera el minuto y medio con un gasto aproximado de un 1 GB de RAM por consulta lo que supera el límite que ofrece Render para su servicio gratuito.

## **Recomendaciones finales**

Aquí te presento algunas recomendaciones para mejorar los modelos presentados en este notebook, que por limitaciones en cuanto al consumo de la RAM no pudieron ser empleados considerando el total de la data.

- **Optimizar los parámetros de TfidfVectorizer**: El modelo utiliza la clase TfidfVectorizer para convertir los textos de los resúmenes en una matriz de características TF-IDF. Se podría experimentar con diferentes valores de los parámetros, como el conjunto de stop words, el rango de n-gramas y los valores de min_df y max_df para encontrar los valores óptimos para el modelo.

- **Considerar el uso de técnicas de reducción de dimensionalidad**: La matriz de características generada por TfidfVectorizer podría tener una alta dimensionalidad, lo que podría afectar negativamente el rendimiento del modelo y aumentar el tiempo de procesamiento. Se podría considerar el uso de técnicas de reducción de dimensionalidad, como PCA o t-SNE, para reducir la dimensionalidad de la matriz de características antes de aplicar el modelo.

- **Evaluar el rendimiento del modelo**: Es importante evaluar el rendimiento del modelo para determinar su capacidad para proporcionar recomendaciones precisas y útiles. Se pueden utilizar técnicas de validación cruzada o de clustering para evaluar la calidad de la agrupación generada por el modelo

## **Modelo empleado en el proyecto**

Es importante tener en cuenta que, debido a las limitaciones de la capacidad de la memoria RAM, el código de los modelos presentados en este notebook podría mejorarse en el futuro. Actualmente, estos son los mejores resultados que podemos mostrar.

La elección del modelo final para este proyecto se basó en su capacidad para adaptarse a las limitaciones de consumo de recursos. Por lo tanto, el modelo seleccionado fue el Modelo1 debido a su menor costo computacional, a pesar de que su precisión es menor que la de otros modelos presentados en este notebook

In [ ]:
# Imprimir las primeras 5 filas del Dataframe "merged"
df[['title', 'overview']].head()

,title,overview
0,Toy Story,led by woody andys toys live happily in his ro...
1,Jumanji,when siblings judy and peter discover an encha...
2,Grumpier Old Men,a family wedding reignites the ancient feud be...
3,Waiting to Exhale,cheated on mistreated and stepped on the women...
4,Father of the Bride Part II,just when george banks has recovered from his ...


In [ ]:
# Imprimir las primeras 5 filas del Dataframe "merged"
df[['title', 'overview_clean']].head()

,title,overview_clean
0,Toy Story,led by woody andys toy live happily in his roo...
1,Jumanji,when sibling judy and peter discover an enchan...
2,Grumpier Old Men,a family wedding reignites the ancient feud be...
3,Waiting to Exhale,cheated on mistreated and stepped on the woman...
4,Father of the Bride Part II,just when george bank ha recovered from his da...


Se tendrán 2 exporaciones para el modelo1, en los cuales 1 tendrá la columna overview y el otro la columna overview_clean. Dado que tienen pesos diferentes el archivo finalmente escogido será aquel que pueda ser desplegado y consumido por render.

In [ ]:
# Se exporta las columnas necesarias para el Sistema de Recomendación
model_1 = df[['title', 'overview']]
model_1.to_csv('movies_recommendations.csv')

In [ ]:
# Se exporta las columnas necesarias para el Sistema de Recomendación
model_2 = df[['title', 'overview_clean']]
model_2.to_csv('movies_recommendations.csv')